In [61]:
from reid_config.paths import PROCESSED_DATA, VAL_MODELS, BEST_MODELS, SOURCE
import os
import cv2
import numpy as np
from openvino.inference_engine import IECore
from tqdm import tqdm
import pickle

In [44]:
def preprocess(path):
    img = cv2.imread(path)
    img = np.expand_dims(np.array(cv2.resize(img, (224, 224)), dtype="float32")/255.0, axis=0)
    img = np.transpose(img, (0, 3, 1, 2))
    return img

class openvino_inf:
    def __init__(self, model_path, weights_path):
        ie = IECore()
        net = ie.read_network(model_path, weights_path)
        self.input_name = [input_name for input_name in net.input_info][0]
        self.output_name = 'StatefulPartitionedCall/vehtype-mn/global_average_pooling2d_1/Mean'
        net.add_outputs([self.output_name])
        self.inf_net = ie.load_network(network=net, device_name='CPU')
    
    def run(self, img):
        return self.inf_net.infer({self.input_name: img})[self.output_name]

In [55]:
DATA_PATH = 'C:\\Users\\A_Pot\\Documents\\WorkProjects\\raw_data\\vdc\\veriwild'

paths = []
idxs = []
idx = 0
for one in os.listdir(DATA_PATH):
    for two in os.listdir(os.path.join(DATA_PATH, one)):
        img_names = os.listdir(os.path.join(DATA_PATH, one, two))
        paths += list(map(lambda x: os.path.join(DATA_PATH, one, two, x), img_names))
        idxs += [idx] * len(img_names) 
        idx += 1
        
len(paths), len(idxs)

(416314, 416314)

In [56]:
model_name = 'vdc_type_model'
model_path = os.path.join(VAL_MODELS, model_name + '.xml')
weights_path = model_path[:-3] + 'bin'

model = openvino_inf(model_path, weights_path)
descriptors = []

for path in tqdm(paths):
    descriptors.append(model.run(preprocess(path)).flatten())

len(descriptors)

100%|█████████████████████████████████████████████████████████████████████████| 416314/416314 [39:23<00:00, 176.15it/s]


416314

In [58]:
descriptors = np.stack(descriptors, axis=0) 
descriptors

(416314, 512)

In [62]:
with open('vdc_type.pickle', 'wb') as f:
    pickle.dump(descriptors, f)